In [ ]:
suppressWarnings(suppressMessages({
    library(Signac)
    library(Seurat)
    library(EnsDb.Mmusculus.v79)
    library(tidyverse)
    library(SingleR)
    library(EnsDb.Hsapiens.v75)
}))

In [ ]:
rds = Read10X_h5("/SGRNJ06/randd/PROJECT/scATAC/project/self_pipe/P23121502_zhongshan/20240102/KP_TAMK_X/02.atac/Result/QC/KP_TAMK_X//KP_TAMK_X_filtered_peak_count.h5")

In [ ]:
rds

In [ ]:
rds = readRDS("/SGRNJ06/randd/PROJECT/scATAC/project/self_pipe/P23121502_zhongshan/20240102/KP_TAMK_X/02.atac/Result/Analysis/KP_TAMK_X//KP_TAMK_X_scATAC_Object.rds")

In [ ]:
rds$ATAC@meta.data

In [ ]:
# merge human mouse 

In [ ]:
# 1. cat合并cell_metrics.tsv, fragments文件
# 2. python处理doublet（重复barcode）

In [2]:
import pandas as pd 

In [3]:
df1 = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/hs/A0824_5_3K_yisheng_wenxian_rapid_T7_6515TESDSTween_TD612_C16/outs/cell_qc_metrics.tsv", sep='\t')
df2 = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/mu/A0824_5_3K_yisheng_wenxian_rapid_T7_6515TESDSTween_TD612_C16/outs/cell_qc_metrics.tsv", sep='\t')

In [ ]:
df = df.drop_duplicates('barcode', keep=False)

In [ ]:
df

In [ ]:
df.to_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/merge/cell_qc_metrics1.tsv", sep='\t', index=False)

In [ ]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/merge/fragments_corrected_dedup_count.tsv.gz",
                                      header=None, sep='\t', names=["chrom", "chromStart", "chromEnd", "barcode", "count"])

In [ ]:
df

In [ ]:
df1 = df.sort_values(['chrom', 'chromStart', 'chromEnd'])

In [ ]:
df1

In [ ]:
df1.to_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/merge/fragments_corrected_dedup_count1.tsv.gz", sep='\t', index=0, header=0)

In [ ]:
library(hdf5r)
library(Seurat)
library(tidyverse)
library(stringr)
library(dplyr)

In [2]:
rds1 = Read10X_h5("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/hs/A0824_5_3K_yisheng_wenxian_rapid_T7_6515TESDSTween_TD612_C16/outs/A0824_5_3K_yisheng_wenxian_rapid_T7_6515TESDSTween_TD612_C16_filtered_peak_count.h5")
rds2 = Read10X_h5("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/mu/A0824_5_3K_yisheng_wenxian_rapid_T7_6515TESDSTween_TD612_C16/outs/A0824_5_3K_yisheng_wenxian_rapid_T7_6515TESDSTween_TD612_C16_filtered_peak_count.h5")

Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' is deprecated; setting repr="T" for you”
as(<dgTMatrix>, "dgCMatrix") is deprecated since Matrix 1.5-0; do as(., "CsparseMatrix") instead

Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' is deprecated; setting repr="T" for you”


In [3]:
dim(rds1)

[1] 135978   3987

In [27]:
dim(rds2)

[1] 123736   6097

In [19]:
df1 = as.data.frame(rds1)
df2 = as.data.frame(rds2)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 4.0 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 5.6 GiB”


In [22]:
df1$shi<-paste("GRCh38","-",sep="")
rownames(df1)<-str_c(df1$shi,rownames(df1))

df2$shi<-paste("GRCm38","-",sep="")
rownames(df2)<-str_c(df2$shi,rownames(df2))

In [25]:
df1 <- select(df1,-shi)
df2 <- select(df2,-shi)

In [30]:
rds1 = as.matrix(df1)
rds2 = as.matrix(df2)

In [ ]:
rds1

In [32]:
c1 <- CreateSeuratObject(counts = rds1, project = "1")

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [33]:
c2 <- CreateSeuratObject(counts = rds2, project = "2")

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [34]:
length(setdiff(c(rownames(c1@meta.data)), c(rownames(c2@meta.data))))

[1] 3523

In [35]:
length(setdiff(c(rownames(c2@meta.data)), c(rownames(c1@meta.data))))

[1] 5633

In [36]:
c3 = (setdiff(c(rownames(c1@meta.data)), c(rownames(c2@meta.data))))
c4 = (setdiff(c(rownames(c2@meta.data)), c(rownames(c1@meta.data))))

In [37]:
c3 = subset(c1, cells=c3)
c4 = subset(c2, cells=c4)

In [38]:
c3

An object of class Seurat 
135978 features across 3523 samples within 1 assay 
Active assay: RNA (135978 features, 0 variable features)

In [39]:
c4

An object of class Seurat 
123736 features across 5633 samples within 1 assay 
Active assay: RNA (123736 features, 0 variable features)

In [40]:
combined <- merge(c3, y = c4,  project = "c")

In [42]:
combined_count = combined@assays$RNA@counts

In [44]:
dim(combined_count)

[1] 259714   9156

[1] 259714   9156

In [45]:
suppressWarnings(suppressMessages({
    library(Signac)
    library(Seurat)
    library(EnsDb.Mmusculus.v79)
    library(tidyverse)
    library(SingleR)
    library(Signac)
    library(Seurat)
    library(GenomicRanges)
    library(future)
}))

In [46]:
counts <- combined_count

In [ ]:
# counts <- Read10X_h5("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/hs/A0824_5_3K_yisheng_wenxian_rapid_T7_6515TESDSTween_TD612_C16/outs/A0824_5_3K_yisheng_wenxian_rapid_T7_6515TESDSTween_TD612_C16_filtered_peak_count.h5")

In [ ]:
metadata <- read.csv(
  file = "/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/merge/cell_qc_metrics1.tsv",
  header = TRUE,
  row.names = 1,
  sep='\t'
)

In [ ]:
counts

In [ ]:
brain_assay <- CreateChromatinAssay(
  counts = counts,
  sep = c("-", "-"),
  genome = "mm10",
  fragments = '/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240111renshu/merge/fragments_corrected_dedup_count1.tsv.gz',
)

In [ ]:
brain <- CreateSeuratObject(
  counts = brain_assay,
  assay = 'peaks',
  project = 'ATAC',
  meta.data = metadata
)

In [ ]:
brain@assays$peaks